In [9]:
!ngrok authtoken 2hnPdR1Nt7RduYxjmn8sNYqR7mV_43f7p1A7TadpTBS9FQABu

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
!pip install pyngrok

In [16]:
%%writefile app.py

from flask import Flask, request, render_template_string
import re
import pandas as pd

app = Flask(__name__)

financial_data = pd.read_excel("K10_filings final.xlsx")

def simple_chatbot(user_query):
    try:
        # Extract company name
        company_pattern = re.compile(r'\b(Microsoft|Apple|Tesla)\b', re.IGNORECASE)
        company_match = company_pattern.search(user_query)
        if not company_match:
            return "Sorry, I couldn't understand the company name in your query. Please ask about Microsoft, Apple, or Tesla."
        company = company_match.group(0).capitalize()

        # Extract year
        year_pattern = re.compile(r'\b(2021|2022|2023)\b')
        year_match = year_pattern.search(user_query)
        if not year_match:
            return "Sorry, I couldn't understand the year in your query. Please ask about 2021, 2022, or 2023."
        year = int(year_match.group(0))

    except Exception as e:
        return f"Sorry, I couldn't understand the query. Error: {str(e)}"

    if "total revenue" in user_query.lower():
        return f"The total revenue of {company} in {year} is ${financial_data[company][year]['Total Revenue']} million."
    elif "net income changed" in user_query.lower():
        current_year = year
        previous_year = year - 1
        change = financial_data[company][current_year]['Net Income'] - financial_data[company][previous_year]['Net Income']
        return f"The net income of {company} has {'increased' if change > 0 else 'decreased'} by ${abs(change)} million over the last year."
    elif "profit margin" in user_query.lower():
        return f"The profit margin of {company} in {year} is {financial_data[company][year]['Profit Margin'] * 100:.2f}%."
    elif "total assets" in user_query.lower():
        return f"The total assets of {company} in {year} are ${financial_data[company][year]['Total Assets']} million."
    elif "liabilities to assets ratio" in user_query.lower():
        return f"The liabilities to assets ratio of {company} in {year} is {financial_data[company][year]['Liabilities to Assets Ratio']:.2f}."
    else:
        return "Sorry, I can only provide information on predefined queries."

@app.route("/", methods=["GET", "POST"])
def index():
    response = ""
    if request.method == "POST":
        user_query = request.form["query"]
        response = simple_chatbot(user_query)
    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Financial Chatbot</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 20px;
                background-color: #f4f4f4;
            }
            .container {
                max-width: 600px;
                margin: auto;
                background: #fff;
                padding: 20px;
                border-radius: 8px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            }
            h1 {
                text-align: center;
            }
            form {
                display: flex;
                flex-direction: column;
            }
            input[type="text"] {
                padding: 10px;
                margin-bottom: 10px;
                border-radius: 4px;
                border: 1px solid #ddd;
                font-size: 16px;
            }
            input[type="submit"] {
                padding: 10px;
                border: none;
                border-radius: 4px;
                background-color: #5cb85c;
                color: #fff;
                font-size: 16px;
                cursor: pointer;
            }
            input[type="submit"]:hover {
                background-color: #4cae4c;
            }
            .response {
                margin-top: 20px;
                padding: 15px;
                background-color: #e9ecef;
                border-radius: 4px;
                font-size: 16px;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Financial Chatbot</h1>
            <form method="post">
                <input type="text" name="query" placeholder="Ask a financial question..." required>
                <input type="submit" value="Ask">
            </form>
            {% if response %}
                <div class="response">{{ response }}</div>
            {% endif %}
        </div>
    </body>
    </html>
    ''', response=response)

if __name__ == "__main__":
    app.run()

Overwriting app.py


In [17]:
from pyngrok import ngrok
import subprocess
import time

# Set your ngrok authtoken
ngrok.set_auth_token("2hnPdR1Nt7RduYxjmn8sNYqR7mV_43f7p1A7TadpTBS9FQABu")

# Run Flask app
subprocess.Popen(['python', 'app.py'])

# Open an ngrok tunnel to the Flask app
http_tunnel = ngrok.connect(5000)
print(f"Public URL: {http_tunnel.public_url}")

# Ensure the Flask app is running
time.sleep(5)

Public URL: https://7cf6-34-74-100-241.ngrok-free.app
